In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil

In [ ]:
optimal_threshold_asc = -5.411945

In [ ]:
# Load predictions
preds_path = ('model_training_checkpoints/model_v33/full_predictions_v33_epoch-10.csv')
preds_df = pd.read_csv(preds_path)

In [ ]:
# Cages included in test dataset
preds_df['file'].str.slice(29, 31).unique().tolist()

In [ ]:
print("hdor_asc")
print("Max score:")
print(preds_df['hdor_asc'].max())
print("Min score:")
print(preds_df['hdor_asc'].min())
print("Clips above threshold:")
print(sum(preds_df['hdor_asc'] >= optimal_threshold_asc))
print("Total clips:")
print(len(preds_df))

In [ ]:
plt.hist(preds_df['hdor_asc'])
plt.show()

In [ ]:
# Filter to positive predictions for hdor_asc
asc_preds_df = preds_df[preds_df['hdor_asc'] >= optimal_threshold_asc]

In [ ]:
# Take random sample of positive predictions
asc_sample_df = asc_preds_df.sample(n=500, random_state=1)

# Remove HPC-specific part of file paths
asc_sample_df['file_path_short'] = asc_sample_df['file'].str.split('/').str[4] + '/' + asc_sample_df['file'].str.split('/').str[5]

# List files in sample to copy to folder
asc_sample_list = asc_sample_df['file_path_short'].tolist()

# Tidy up columns and indexes
asc_sample_df.set_index(['file_path_short', 'start_time', 'end_time'], inplace=True)
asc_sample_df = asc_sample_df[['hdor_asc', 'hdor_arch', 'hdor_all']]

In [ ]:
# Save sample file list to csv
asc_sample_df.to_csv('preds_tdl/_scores.csv')

In [ ]:
new_filename = asc_sample_list[0].split('/')[0] + '-' + asc_sample_list[0].split('/')[1]

In [ ]:
# Copy selected files to audio folder
# Results in <500 files as some files contain multiple sampled clips
usb_dir = '/mnt/d'
save_dir = 'preds_tdl/audio'
os.makedirs(save_dir, exist_ok=True)

for file_path in asc_sample_list:
    usb_path = usb_dir + '/' + file_path
    new_filename = file_path.split('/')[0] + '-' + file_path.split('/')[1]
    save_path = save_dir + '/' + new_filename
    shutil.copyfile(usb_path, save_path)